In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
import math
import warnings
warnings.filterwarnings("ignore")
import os

In [2]:
# UK Paths
APC_DIR = r'Q:\Data\Observed\Transit\Muni\APC\CMP_2021'

CMPplus_DIR = r'Q:\CMP\LOS Monitoring 2021\CMP_plus_shp\old_cmp_plus'
INRIX_DIR = r'Q:\GIS\Transportation\Roads\INRIX\XD\21_01\maprelease-shapefiles\SF'  # Add INRIX street names to be more comprehensive
NETCONF_DIR = r'Q:\CMP\LOS Monitoring 2021\Network_Conflation'
OUT_DIR = r'Q:\CMP\LOS Monitoring 2021\Transit\Speed\test'

# Read in transit stop file downloaded from GTFS
stops = gp.read_file(os.path.join(APC_DIR, 'GTFS_stops.shp'))

# Read in transit APC data
apc = pd.read_csv(os.path.join(APC_DIR, 'APC_STOP_BOARDINGS_FACTS_SFCTACMP2021.csv'))

# Read in the list of stop pairs manually identified that are patially overlap with cmp segments
overlap_pairs = pd.read_csv(os.path.join(APC_DIR, 'Postprocessing_overlapping_transit_segments.csv'))
                  
apc_notnull = apc[pd.notnull(apc['CLOSE_DATE_TIME'])]
print('Percent of records with null value: ', round(100 - 100*(len(apc_notnull)/len(apc)),2))
del apc

Percent of records with null value:  62.57


In [206]:
apc = pd.read_csv(os.path.join(APC_DIR, 'APC_STOP_BOARDINGS_FACTS_SFCTACMP2021_preCOVID.csv'))
apc_notnull = apc[pd.notnull(apc['CLOSE_DATE_TIME'])]
print('Percent of records with null value: ', round(100 - 100*(len(apc_notnull)/len(apc)),2))
del apc

OUT_DIR = r'Q:\CMP\LOS Monitoring 2021\Transit\Speed\2020'
overlap_pairs = pd.read_csv(os.path.join(APC_DIR, 'Postprocessing_overlapping_transit_segments_BayBridge_v2.csv'))

Percent of records with null value:  51.75


In [2]:
# APC_DIR = r'Q:\Data\Observed\Transit\Muni\APC\2022\04_Apr'
# apc = pd.read_csv(os.path.join(APC_DIR, 'apc_stop_boardings_facts_SFCTA_2022-04.csv'))

# APC_DIR = r'Q:\Data\Observed\Transit\Muni\APC\2022\05_May'
# apc = pd.read_csv(os.path.join(APC_DIR, 'apc_stop_boardings_facts_SFCTA_2022-05.csv'))

APC_DIR = r'Q:\Data\Observed\Transit\Muni\APC\2022\06_Jun'
apc = pd.read_csv(os.path.join(APC_DIR, 'apc_stop_boardings_facts_SFCTA_2022-06.csv'))

apc_notnull = apc[pd.notnull(apc['CLOSE_DATE_TIME'])]
print('Percent of records with null value: ', round(100 - 100*(len(apc_notnull)/len(apc)),2))
del apc

Percent of records with null value:  46.36


In [244]:
overlap_pairs = pd.read_csv(os.path.join(APC_DIR, 'Postprocessing_overlapping_transit_segments_BayBridge_v2.csv'))

In [207]:
overlap_pairs = pd.read_csv(os.path.join(APC_DIR, 'Postprocessing_overlapping_transit_segments_BayBridge.csv'))

In [3]:
apc_notnull['CLOSE_DATE_TIME']

1          2022-06-01 04:59:39
2          2022-06-01 23:50:15
3          2022-06-01 23:51:48
4          2022-06-01 23:55:52
8          2022-06-01 23:57:06
                  ...         
7350117    2022-06-30 03:03:55
7350124    2022-06-30 03:27:47
7350125    2022-06-30 02:48:54
7350126    2022-06-30 02:49:59
7350136    2022-06-30 02:56:09
Name: CLOSE_DATE_TIME, Length: 3942433, dtype: object

In [4]:
apc_notnull['CLOSE_DATE_TIME'].str[-2:]

1          39
2          15
3          48
4          52
8          06
           ..
7350117    55
7350124    47
7350125    54
7350126    59
7350136    09
Name: CLOSE_DATE_TIME, Length: 3942433, dtype: object

In [5]:
apc_notnull['Date']=pd.to_datetime(apc_notnull['ACTUALDATE'])
apc_notnull['DOW']=apc_notnull.Date.dt.dayofweek #Mon-0
apc_notnull['DayType'] = np.where(apc_notnull['DOW']<=4, 'Weekdays', np.where(apc_notnull['DOW']==5, 'Saturday', 'Sunday'))
apc_notnull['Month']=apc_notnull.Date.dt.month

apc_notnull['Close_Hour'] = apc_notnull['CLOSE_DATE_TIME'].str[10:13].astype(int)
apc_notnull['Close_Minute'] = apc_notnull['CLOSE_DATE_TIME'].str[14:16].astype(int)
apc_notnull['Close_Second'] = apc_notnull['CLOSE_DATE_TIME'].str[17:19].astype(int)
# apc_notnull['Close_Period'] = apc_notnull['CLOSE_DATE_TIME'].str[-2:]
apc_notnull['Close_Time'] = apc_notnull['Date'].astype('str') + ' ' + apc_notnull['Close_Hour'].astype('str') + ':' + apc_notnull['Close_Minute'].astype('str') + ':' + apc_notnull['Close_Second'].astype('str')
apc_notnull['Close_Time'] = pd.to_datetime(apc_notnull['Close_Time'])
apc_notnull['Epoch'] = 2*apc_notnull['Close_Time'].dt.hour + apc_notnull['Close_Time'].dt.minute//30 

apc_notnull['Open_Hour'] = apc_notnull['OPEN_DATE_TIME'].str[10:13].astype(int)
apc_notnull['Open_Minute'] = apc_notnull['OPEN_DATE_TIME'].str[14:16].astype(int)
apc_notnull['Open_Second'] = apc_notnull['OPEN_DATE_TIME'].str[17:19].astype(int)
# apc_notnull['Open_Period'] = apc_notnull['OPEN_DATE_TIME'].str[-2:]
apc_notnull['Open_Time'] = apc_notnull['Date'].astype('str') + ' ' + apc_notnull['Open_Hour'].astype('str') + ':' + apc_notnull['Open_Minute'].astype('str') + ':' + apc_notnull['Open_Second'].astype('str')
apc_notnull['Open_Time'] = pd.to_datetime(apc_notnull['Open_Time'])

In [248]:
apc_notnull['Date']=pd.to_datetime(apc_notnull['ACTUALDATE'])
apc_notnull['DOW']=apc_notnull.Date.dt.dayofweek #Mon-0
apc_notnull['DayType'] = np.where(apc_notnull['DOW']<=4, 'Weekdays', np.where(apc_notnull['DOW']==5, 'Saturday', 'Sunday'))
apc_notnull['Month']=apc_notnull.Date.dt.month

apc_notnull['Close_Hour'] = apc_notnull['CLOSE_DATE_TIME'].str[10:12].astype(int)
apc_notnull['Close_Minute'] = apc_notnull['CLOSE_DATE_TIME'].str[13:15].astype(int)
apc_notnull['Close_Second'] = apc_notnull['CLOSE_DATE_TIME'].str[16:18].astype(int)
apc_notnull['Close_Period'] = apc_notnull['CLOSE_DATE_TIME'].str[-2:]
apc_notnull['Close_Time'] = apc_notnull['Date'].astype('str') + ' ' + apc_notnull['Close_Hour'].astype('str') + ':' + apc_notnull['Close_Minute'].astype('str') + ':' + apc_notnull['Close_Second'].astype('str') + ' ' + apc_notnull['Close_Period']
apc_notnull['Close_Time'] = pd.to_datetime(apc_notnull['Close_Time'])
apc_notnull['Epoch'] = 2*apc_notnull['Close_Time'].dt.hour + apc_notnull['Close_Time'].dt.minute//30 

apc_notnull['Open_Hour'] = apc_notnull['OPEN_DATE_TIME'].str[10:12].astype(int)
apc_notnull['Open_Minute'] = apc_notnull['OPEN_DATE_TIME'].str[13:15].astype(int)
apc_notnull['Open_Second'] = apc_notnull['OPEN_DATE_TIME'].str[16:18].astype(int)
apc_notnull['Open_Period'] = apc_notnull['OPEN_DATE_TIME'].str[-2:]
apc_notnull['Open_Time'] = apc_notnull['Date'].astype('str') + ' ' + apc_notnull['Open_Hour'].astype('str') + ':' + apc_notnull['Open_Minute'].astype('str') + ':' + apc_notnull['Open_Second'].astype('str') + ' ' + apc_notnull['Open_Period']
apc_notnull['Open_Time'] = pd.to_datetime(apc_notnull['Open_Time'])

ValueError: invalid literal for int() with base 10: ':5'

In [9]:
apc_notnull.columns

Index(['ACTUALDATE', 'SERVICE_DAY_OF_WEEK', 'PRIMARY_KEY',
       'EFFECTIVE_DATE_KEY', 'DAY_TYPE', 'BOOKING_ID', 'BLOCK_NUM_EXTERNAL',
       'BLOCK_NUM', 'TRAP_BUS_TYPE_ABBR', 'DIVISION_NAME',
       'SPECIAL_TRIP_OPTION_ID', 'ROUTE_ALPHA', 'DIRECTION_CODE_ID',
       'DIRECTION', 'VARIATION', 'TRIP_ID_EXTERNAL', 'TRIP_ID', 'START_TPID',
       'END_TPID', 'START_LAYOVER', 'END_LAYOVER', 'START_TIME_RAW',
       'END_TIME_RAW', 'START_TIME', 'END_TIME', 'TRIP_START_TIME',
       'TRIP_START_HOUR', 'TOTAL_TRIP_DISTANCE', 'SCHED_RUNNING_TIME_SECONDS',
       'STOP_LINE_SEQUENCE', 'BS_ID', 'IS_TERMINAL_INITIAL', 'ACCUM_DISTANCE',
       'SCHEDULE_LOAD_DATE', 'APC_SOURCE', 'VEHICLE_ID', 'ACTUAL_SEQUENCE',
       'RUN_ID', 'OPERATOR_ID', 'STOP_ARRIVAL_HOUR', 'OPEN_DATE_TIME',
       'CLOSE_DATE_TIME', 'DWELL_TIME', 'ONS', 'OFFS', 'MAX_LOAD',
       'REV_DISTANCE', 'NON_REV_DISTANCE', 'REV_SECONDS', 'NON_REV_SECONDS',
       'PASSENGER_MILES', 'PASSENGER_HOURS', 'VEH_LAT', 'VEH_LONG',
    

In [6]:
apc_notnull['ROUTE_ALPHA'].unique()

array([1, 5, '005', '005R', '007', 7, 8, '008', '008AX', '008BX', '009',
       9, '009R', '012', 12, 14, '014', '014R', '015', 15, 18, 19, 22, 23,
       24, 25, 27, 28, 29, 30, 31, 33, 38, '038', '038R', '043', 43, 44,
       45, 48, 49, 52, 54, 55, 57, 58, 66, 67, 90, 91, '091', '714',
       'L-OWL', 'N-OWL'], dtype=object)

In [14]:
apc_notnull.loc[apc_notnull['ROUTE_ALPHA']==25, ['ACTUALDATE','BS_ID','STOP_LINE_SEQUENCE','Close_Time','Open_Time','REV_DISTANCE']].head()

,ACTUALDATE,BS_ID,STOP_LINE_SEQUENCE,Close_Time,Open_Time,REV_DISTANCE
3602625,2022-06-01 00:00:00,7948,1,2022-06-01 12:29:44,2022-06-01 11:21:46,4.421809
3602628,2022-06-01 00:00:00,3229,5,2022-06-01 13:28:25,2022-06-01 12:41:38,1.068527
3602633,2022-06-01 00:00:00,7558,10,2022-06-01 13:31:56,2022-06-01 13:31:45,0.000000
3602635,2022-06-01 00:00:00,4936,2,2022-06-01 10:03:15,2022-06-01 10:03:01,0.104122
3602636,2022-06-01 00:00:00,4934,3,2022-06-01 10:03:55,2022-06-01 10:03:47,0.163155


In [251]:
apc_notnull[apc_notnull['ROUTE_ALPHA']=='025'].head()

,ACTUALDATE,SERVICE_DAY_OF_WEEK,PRIMARY_KEY,EFFECTIVE_DATE_KEY,DAY_TYPE,BOOKING_ID,BLOCK_NUM_EXTERNAL,BLOCK_NUM,TRAP_BUS_TYPE_ABBR,DIVISION_NAME,...,CALC_SPEED,IS_TRIP_SAMPLED,STOPS_SAMPLED,ACTIVITY_LOAD_DATE,EXPORT_DATE,Date,DOW,DayType,Month,Close_Hour


In [8]:
apc_notnull.loc[apc_notnull['ROUTE_ALPHA'].isin([25]), 'BS_ID'].unique()

array([7948, 3229, 7558, 4936, 4934, 4935, 3670, 8018, 8019, 8017, 8020,
       8021, 7917, 7949, 7556, 7557], dtype=int64)

In [214]:
apc_notnull.loc[apc_notnull['ROUTE_ALPHA'].isin(['025']), 'BS_ID'].unique()

array([7917, 3670, 7004, 7949, 8001, 3229, 3673, 3674, 3675, 3904, 4934,
       4935, 4936, 7558, 7560, 7590, 7948, 3905, 7556, 8000, 7557, 7555],
      dtype=int64)

In [12]:
stops = gp.read_file(os.path.join(APC_DIR, 'GTFS_stops.shp'))

In [7]:
#Define WGS 1984 coordinate system
wgs84 = {'proj': 'longlat', 'ellps': 'WGS84', 'datum': 'WGS84', 'no_defs': True}
#Define NAD 1983 StatePlane California III
cal3 = {'proj': 'lcc +lat_1=37.06666666666667 +lat_2=38.43333333333333 +lat_0=36.5 +lon_0=-120.5 +x_0=2000000 +y_0=500000.0000000002', 'ellps': 'GRS80', 'datum': 'NAD83', 'no_defs': True}

# CMP network
cmp_segs_org = gp.read_file(os.path.join(CMPplus_DIR, 'cmp_segments_plus.shp'))
cmp_segs_prj = cmp_segs_org.to_crs(cal3)
cmp_segs_prj['cmp_name'] = cmp_segs_prj['cmp_name'].str.replace('/ ','/')
cmp_segs_prj['cmp_name'] = cmp_segs_prj['cmp_name'].str.lower()
cmp_segs_prj['Length'] = cmp_segs_prj.geometry.length 
cmp_segs_prj['Length'] = cmp_segs_prj['Length'] * 3.2808  #meters to feet

# INRIX network
inrix_net=gp.read_file(os.path.join(INRIX_DIR, 'Inrix_XD_2101_SF_manualedit.shp'))
inrix_net['RoadName'] = inrix_net['RoadName'].str.lower()

cmp_inrix_corr = pd.read_csv(os.path.join(NETCONF_DIR, 'CMP_Segment_INRIX_Links_Correspondence_2101_Manual_PLUS_Updated.csv'))

# Convert transit stops original coordinate system to state plane
stops = stops.to_crs(cal3)
stops['stop_name'] = stops['stop_name'].str.lower()
stops[['street_1','street_2']] = stops.stop_name.str.split('&', expand=True)  #split stop name into operating street and intersecting street


# Create a buffer zone for each cmp segment
ft=160   # According to the memo from last CMP cycle
mt=round(ft/3.2808,4)
stops_buffer=stops.copy()
stops_buffer['geometry'] = stops_buffer.geometry.buffer(mt)
#stops_buffer.to_file(os.path.join(MAIN_DIR, 'stops_buffer.shp'))

# cmp segments intersecting transit stop buffer zone
cmp_segs_intersect=gp.sjoin(cmp_segs_prj, stops_buffer, op='intersects').reset_index()

In [21]:
stops['near_cmp'] = 0
cmp_segs_intersect['name_match']=0
for stop_idx in range(len(stops)):
    stop_id = stops.loc[stop_idx, 'stop_id']
    stop_geo = stops.loc[stop_idx]['geometry']
    stop_names = stops.loc[stop_idx]['street_1'].split('/')
    if 'point lobos' in stop_names:
        stop_names = stop_names + ['geary']
    if stop_id == 7357:
        stop_names = stop_names + ['third st']
    
    cmp_segs_int = cmp_segs_intersect[cmp_segs_intersect['stop_id']==stop_id]
    cmp_segs_idx = cmp_segs_intersect.index[cmp_segs_intersect['stop_id']==stop_id].tolist()
    
    near_dis = 5000
    if ~cmp_segs_int.empty:
        for seg_idx in cmp_segs_idx:
            cmp_seg_id = cmp_segs_int.loc[seg_idx, 'cmp_segid']
            cmp_seg_geo = cmp_segs_int.loc[seg_idx]['geometry']
            cmp_seg_names = cmp_segs_int.loc[seg_idx]['cmp_name'].split('/')
            if 'bayshore' in cmp_seg_names:
                cmp_seg_names = cmp_seg_names + ['bay shore']
            if '3rd st' in cmp_seg_names:
                cmp_seg_names = cmp_seg_names + ['third st']
            if '19th ave' in cmp_seg_names:
                cmp_seg_names = cmp_seg_names + ['19th avenue']
            if 'geary' in cmp_seg_names:
                cmp_seg_names = cmp_seg_names + ['point lobos']
                
            
            # Add INRIX street name to be comprehensive
            inrix_links = cmp_inrix_corr[cmp_inrix_corr['CMP_SegID']==cmp_seg_id]
            inrix_link_names = inrix_net[inrix_net['XDSegID'].isin(inrix_links['INRIX_SegID'])]['RoadName'].tolist()
            inrix_link_names = list(filter(None, inrix_link_names)) 
            if len(inrix_link_names) > 0:
                inrix_link_names = list(set(inrix_link_names))
                cmp_seg_link_names = cmp_seg_names + inrix_link_names
            else:
                cmp_seg_link_names = cmp_seg_names
            
            matched_names= [stop_name for stop_name in stop_names if any(cname in stop_name for cname in cmp_seg_link_names)]
            if len(matched_names)>0:
                stops.loc[stop_idx, 'near_cmp']=1
                cmp_segs_intersect.loc[seg_idx, 'name_match']=1
                cur_dis = stop_geo.distance(cmp_seg_geo)
                cmp_segs_intersect.loc[seg_idx, 'distance']=cur_dis
                near_dis = min(near_dis, cur_dis)
                stops.loc[stop_idx, 'near_dis']=near_dis

stops_near_cmp = stops[stops['near_cmp']==1]
stops_near_cmp = stops_near_cmp.to_crs(wgs84)

stops_near_cmp_list = stops_near_cmp['stop_id'].unique().tolist()

cmp_segs_near = cmp_segs_intersect[cmp_segs_intersect['name_match']==1]

# Remove mismatched stops based on manual review
remove_cmp_stop = [(175, 5546), (175, 5547), (175, 7299), (175, 5544),
                  (66, 7744),
                  (214, 7235),
                  (107, 4735),
                  (115, 4275),
                  (143, 4824),
                  (172, 5603)]
for remove_idx in range(len(remove_cmp_stop)):
    rmv_cmp_id = remove_cmp_stop[remove_idx][0]
    rmv_stop_id = remove_cmp_stop[remove_idx][1]
    remove_df_idx = cmp_segs_near.index[(cmp_segs_near['cmp_segid']==rmv_cmp_id) & (cmp_segs_near['stop_id']==rmv_stop_id)].tolist()
    if len(remove_df_idx) > 0:
        cmp_segs_near = cmp_segs_near.drop([remove_df_idx[0]], axis=0)

In [8]:
apc_cmp = apc_notnull[(apc_notnull['DOW'] ==1) | (apc_notnull['DOW'] ==2) | (apc_notnull['DOW'] ==3)]  # only Tue, Wed, and Thu
apc_cmp['Day']=apc_cmp.Date.dt.day
apc_cmp = apc_cmp[((apc_cmp['Month']==4) & (apc_cmp['Day'] > 5)) | ((apc_cmp['Month']==5) & (apc_cmp['Day'] < 21))] # CMP monitoring dates
apc_cmp['Open_Time_float'] = apc_cmp['Open_Hour'] + apc_cmp['Open_Minute']/60 + apc_cmp['Open_Second']/3600
apc_cmp['Close_Time_float'] = apc_cmp['Close_Hour'] + apc_cmp['Close_Minute']/60 + apc_cmp['Close_Second']/3600

In [246]:
# # Match AM&PM transit stops to CMP segments
angle_thrd = 10
def match_stop_pairs_to_cmp(apc_cmp_df, stops_near_cmp_list, cmp_segs_near, cmp_segs_prj, angle_thrd):
    apc_pairs_df = pd.DataFrame()
    pair_cnt = 0
    for cur_stop_idx in range(5000):  #Enumerate the records in apc_cmp_df
        # Check if the stop_id is in the previously matched list
        if apc_cmp_df.loc[cur_stop_idx, 'stop_id'] in stops_near_cmp_list:
            next_stop_match = 0
            if apc_cmp_df.loc[cur_stop_idx + 1, 'stop_id'] in stops_near_cmp_list:
                next_stop_idx = cur_stop_idx + 1 
                next_stop_match = 1
            # This is to ensure the stop_id associated with cur_stop_idx + 2 is also checked when the stop_id associated with cur_stop_idx + 1 is not found in the matched list
            elif apc_cmp_df.loc[cur_stop_idx + 2, 'stop_id'] in stops_near_cmp_list:
                next_stop_idx = cur_stop_idx +2
                next_stop_match = 2

            if next_stop_match >0:
                # Transit stop of interest
                cur_stop_trip_id = apc_cmp_df.loc[cur_stop_idx, 'TRIP_ID_EXTERNAL']
                cur_stop_date = apc_cmp_df.loc[cur_stop_idx, 'Date']
                cur_stop_veh_id = apc_cmp_df.loc[cur_stop_idx, 'VEHICLE_ID']
                cur_stop_route_alpha = apc_cmp_df.loc[cur_stop_idx, 'ROUTE_ALPHA']
                cur_stop_route_dir = apc_cmp_df.loc[cur_stop_idx, 'DIRECTION']

                # Succeeding candidate stop in the dataframe
                next_stop_trip_id = apc_cmp_df.loc[next_stop_idx, 'TRIP_ID_EXTERNAL']
                next_stop_date = apc_cmp_df.loc[next_stop_idx, 'Date']
                next_stop_veh_id = apc_cmp_df.loc[next_stop_idx, 'VEHICLE_ID']

                # Check if two stops share the same trip id, date, and vehicle id
                if (cur_stop_trip_id == next_stop_trip_id) & (cur_stop_date == next_stop_date) & (cur_stop_veh_id == next_stop_veh_id):

                    cur_stop_id = apc_cmp_df.loc[cur_stop_idx, 'stop_id']
                    cur_stop_open_time = apc_cmp_df.loc[cur_stop_idx, 'Open_Time']
                    cur_stop_close_time = apc_cmp_df.loc[cur_stop_idx, 'Close_Time']
                    cur_stop_dwell_time = apc_cmp_df.loc[cur_stop_idx, 'DWELL_TIME']

                    next_stop_id = apc_cmp_df.loc[next_stop_idx, 'stop_id']
                    next_stop_open_time = apc_cmp_df.loc[next_stop_idx, 'Open_Time']
                    next_stop_close_time = apc_cmp_df.loc[next_stop_idx, 'Close_Time']
                    next_stop_dwell_time = apc_cmp_df.loc[next_stop_idx, 'DWELL_TIME']

                    # Matched CMP segments for the current stop
                    cur_stop_near_segs = list(cmp_segs_near[cmp_segs_near['stop_id']==cur_stop_id]['cmp_segid'])
                    # Matched CMP segments for the succeeding stop
                    next_stop_near_segs = list(cmp_segs_near[cmp_segs_near['stop_id']==next_stop_id]['cmp_segid'])

                    # Find the common CMP segments in two sets
                    common_segs = list(set(cur_stop_near_segs) & set(next_stop_near_segs))
                    
                    if len(common_segs)>0:
                        cur_stop_geo = apc_cmp_df.loc[cur_stop_idx]['geometry']   # location geometry of current stop
                        next_stop_geo = apc_cmp_df.loc[next_stop_idx]['geometry']  # location geometry of succeeding stop

                        # Traveling direction from current stop to succeeding stop
                        stop_degree=(180 / math.pi) * math.atan2(next_stop_geo.x - cur_stop_geo.x, next_stop_geo.y - cur_stop_geo.y)

                        for common_seg_id in common_segs: # Iterate through the common segments set
                            common_seg_geo = cmp_segs_prj[cmp_segs_prj['cmp_segid']==common_seg_id]['geometry']  # line geometry of the cmp segment

                            cur_stop_dis = common_seg_geo.project(cur_stop_geo)  # distance between the beginning point of cmp segment and the projected transit stop
                            cur_stop_projected = common_seg_geo.interpolate(cur_stop_dis) #project current stop onto cmp segment
                            cur_stop_loc = cur_stop_dis * 3.2808  #meters to feet

                            next_stop_dis = common_seg_geo.project(next_stop_geo)
                            next_stop_projected = common_seg_geo.interpolate(next_stop_dis) #project next stop onto cmp segment
                            next_stop_loc = next_stop_dis * 3.2808  #meters to feet

                            # Ensure the degree is calculated following the traveling direction
                            if float(next_stop_loc) > float(cur_stop_loc):
                                cmp_degree=(180 / math.pi) * math.atan2(next_stop_projected.x - cur_stop_projected.x, next_stop_projected.y - cur_stop_projected.y)
                            else:
                                cmp_degree=(180 / math.pi) * math.atan2(cur_stop_projected.x - next_stop_projected.x, cur_stop_projected.y - next_stop_projected.y)

                            stop_cmp_angle = abs(cmp_degree-stop_degree)
                            if stop_cmp_angle>270:
                                stop_cmp_angle=360-stop_cmp_angle

                            if stop_cmp_angle < angle_thrd:   # Angle between CMP segment and transit stop pair meets the requirement
                                cur_next_traveltime = (next_stop_open_time-cur_stop_open_time).total_seconds()
                                cur_next_loc_dis = abs(float(next_stop_loc) - float(cur_stop_loc)) / 5280  #feet to miles

                                # Remove records deemed erroneous according to memo from last cycle
                                if (cur_next_loc_dis>0) & (cur_next_traveltime>0):
                                    if 3600*cur_next_loc_dis/cur_next_traveltime<=55:
                                        # Add matched stop pairs to the new dataframe for subsequent speed calculation
                                        apc_pairs_df.loc[pair_cnt, 'cmp_segid'] = common_seg_id
                                        apc_pairs_df.loc[pair_cnt, 'trip_id'] = cur_stop_trip_id
                                        apc_pairs_df.loc[pair_cnt, 'trip_date'] = cur_stop_date
                                        apc_pairs_df.loc[pair_cnt, 'vehicle_id'] = cur_stop_veh_id
                                        apc_pairs_df.loc[pair_cnt, 'route_alpha'] = cur_stop_route_alpha
                                        apc_pairs_df.loc[pair_cnt, 'direction'] = cur_stop_route_dir

                                        apc_pairs_df.loc[pair_cnt, 'cur_stop_id'] = cur_stop_id
                                        apc_pairs_df.loc[pair_cnt, 'cur_stop_open_time'] = cur_stop_open_time
                                        apc_pairs_df.loc[pair_cnt, 'cur_stop_close_time'] = cur_stop_close_time
                                        apc_pairs_df.loc[pair_cnt, 'cur_stop_dwell_time'] = cur_stop_dwell_time
                                        apc_pairs_df.loc[pair_cnt, 'cur_stop_loc'] = float(cur_stop_loc)

                                        apc_pairs_df.loc[pair_cnt, 'next_stop_id'] = next_stop_id
                                        apc_pairs_df.loc[pair_cnt, 'next_stop_open_time'] = next_stop_open_time
                                        apc_pairs_df.loc[pair_cnt, 'next_stop_close_time'] = next_stop_close_time
                                        apc_pairs_df.loc[pair_cnt, 'next_stop_dwell_time'] = next_stop_dwell_time
                                        apc_pairs_df.loc[pair_cnt, 'next_stop_loc'] = float(next_stop_loc)

                                        apc_pairs_df.loc[pair_cnt, 'cur_next_time'] = cur_next_traveltime
                                        apc_pairs_df.loc[pair_cnt, 'cur_next_loc_dis'] = cur_next_loc_dis
                                        if next_stop_match == 1: 
                                            apc_pairs_df.loc[pair_cnt, 'cur_next_rev_dis'] = apc_cmp_df.loc[cur_stop_idx, 'REV_DISTANCE']
                                        else:
                                            apc_pairs_df.loc[pair_cnt, 'cur_next_rev_dis'] = apc_cmp_df.loc[cur_stop_idx, 'REV_DISTANCE'] + apc_cmp_df.loc[cur_stop_idx + 1, 'REV_DISTANCE']

                                        pair_cnt = pair_cnt + 1

        if cur_stop_idx % 50000 == 0:
            print('Processed %s percent' % round(100*cur_stop_idx/len(apc_cmp_df), 2))
        
    # Update the stop location for CMP 175 due to its irregular geometry
    apc_pairs_df['cur_stop_loc'] = np.where(apc_pairs_df['cmp_segid']==175,
                                         np.where(apc_pairs_df['cur_stop_id']==5543, 173.935,
                                                 np.where(apc_pairs_df['cur_stop_id']==5545, 1020.629,
                                                         np.where(apc_pairs_df['cur_stop_id']==5836, 1804.72,
                                                                 np.where(apc_pairs_df['cur_stop_id']==5835, 2685.807, apc_pairs_df['cur_stop_loc'])))),
                                          apc_pairs_df['cur_stop_loc'])
    apc_pairs_df['next_stop_loc'] = np.where(apc_pairs_df['cmp_segid']==175,
                                         np.where(apc_pairs_df['next_stop_id']==5543, 173.935,
                                                 np.where(apc_pairs_df['next_stop_id']==5545, 1020.629,
                                                         np.where(apc_pairs_df['next_stop_id']==5836, 1804.72,
                                                                 np.where(apc_pairs_df['next_stop_id']==5835, 2685.807, apc_pairs_df['next_stop_loc'])))),
                                          apc_pairs_df['next_stop_loc'])
    apc_pairs_df['cur_next_loc_dis'] = np.where(apc_pairs_df['cmp_segid']==175,
                                         abs(apc_pairs_df['next_stop_loc'] - apc_pairs_df['cur_stop_loc']) / 5280,
                                          apc_pairs_df['cur_next_loc_dis'])
    return apc_pairs_df
    
# ## AM 
# apc_cmp_am = apc_cmp[(apc_cmp['Close_Period']=='AM') & (apc_cmp['Open_Hour']<9) & (apc_cmp['Close_Hour']>6)]
# apc_cmp_am = apc_cmp_am.merge(stops, left_on='BS_ID', right_on='stop_id', how='left')
# apc_cmp_am = apc_cmp_am.sort_values(by=['TRIP_ID_EXTERNAL', 'Date', 'VEHICLE_ID', 'Open_Time']).reset_index()

print('------------Start processing AM trips------------')
apc_pairs_am = match_stop_pairs_to_cmp(apc_cmp_am, stops_near_cmp_list, cmp_segs_near, cmp_segs_prj, angle_thrd)
                                      
# ## PM 
# apc_cmp_pm = apc_cmp[(apc_cmp['Close_Period']=='PM') & ((apc_cmp['Open_Time_float']<=6.5) & (apc_cmp['Close_Time_float']>=4.5))]

# apc_cmp_pm = apc_cmp_pm.merge(stops, left_on='BS_ID', right_on='stop_id', how='left')
# apc_cmp_pm = apc_cmp_pm.sort_values(by=['TRIP_ID_EXTERNAL', 'Date', 'VEHICLE_ID', 'Open_Time']).reset_index()

print('------------Start processing PM trips------------')
apc_pairs_pm = match_stop_pairs_to_cmp(apc_cmp_pm, stops_near_cmp_list, cmp_segs_near, cmp_segs_prj, angle_thrd)

------------Start processing AM trips------------
Processed 0.0 percent


KeyError: 'cmp_segid'

In [10]:
apc_cmp = apc_notnull[(apc_notnull['DOW'] ==1) | (apc_notnull['DOW'] ==2) | (apc_notnull['DOW'] ==3)]  # only Tue, Wed, and Thu
apc_cmp['Day']=apc_cmp.Date.dt.day
# apc_cmp = apc_cmp[((apc_cmp['Month']==4) & (apc_cmp['Day'] > 5)) | ((apc_cmp['Month']==5) & (apc_cmp['Day'] < 21))] # CMP monitoring dates
apc_cmp['Open_Time_float'] = apc_cmp['Open_Hour'] + apc_cmp['Open_Minute']/60 + apc_cmp['Open_Second']/3600
apc_cmp['Close_Time_float'] = apc_cmp['Close_Hour'] + apc_cmp['Close_Minute']/60 + apc_cmp['Close_Second']/3600

# ## AM 
apc_cmp_am = apc_cmp[(apc_cmp['Close_Period']=='AM') & (apc_cmp['Open_Hour']<9) & (apc_cmp['Close_Hour']>6)]
apc_cmp_am = apc_cmp_am.merge(stops, left_on='BS_ID', right_on='stop_id', how='left')
apc_cmp_am = apc_cmp_am.sort_values(by=['TRIP_ID_EXTERNAL', 'Date', 'VEHICLE_ID', 'Open_Time']).reset_index()
                                   
# ## PM 
apc_cmp_pm = apc_cmp[(apc_cmp['Close_Period']=='PM') & ((apc_cmp['Open_Time_float']<=6.5) & (apc_cmp['Close_Time_float']>=4.5))]
apc_cmp_pm = apc_cmp_pm.merge(stops, left_on='BS_ID', right_on='stop_id', how='left')
apc_cmp_pm = apc_cmp_pm.sort_values(by=['TRIP_ID_EXTERNAL', 'Date', 'VEHICLE_ID', 'Open_Time']).reset_index()

KeyError: 'Close_Period'

In [13]:
apc_cmp = apc_notnull[(apc_notnull['DOW'] ==1) | (apc_notnull['DOW'] ==2) | (apc_notnull['DOW'] ==3)]  # only Tue, Wed, and Thu
apc_cmp['Day']=apc_cmp.Date.dt.day
# apc_cmp = apc_cmp[((apc_cmp['Month']==4) & (apc_cmp['Day'] > 5)) | ((apc_cmp['Month']==5) & (apc_cmp['Day'] < 21))] # CMP monitoring dates
apc_cmp['Open_Time_float'] = apc_cmp['Open_Hour'] + apc_cmp['Open_Minute']/60 + apc_cmp['Open_Second']/3600
apc_cmp['Close_Time_float'] = apc_cmp['Close_Hour'] + apc_cmp['Close_Minute']/60 + apc_cmp['Close_Second']/3600

# ## AM 
apc_cmp_am = apc_cmp[(apc_cmp['Open_Hour']<9) & (apc_cmp['Close_Hour']>6)]
apc_cmp_am = apc_cmp_am.merge(stops, left_on='BS_ID', right_on='stop_id', how='left')
apc_cmp_am = apc_cmp_am.sort_values(by=['TRIP_ID_EXTERNAL', 'Date', 'VEHICLE_ID', 'Open_Time']).reset_index()
                                   
# ## PM 
apc_cmp_pm = apc_cmp[(apc_cmp['Open_Time_float']<=18.5) & (apc_cmp['Close_Time_float']>=16.5)]
apc_cmp_pm = apc_cmp_pm.merge(stops, left_on='BS_ID', right_on='stop_id', how='left')
apc_cmp_pm = apc_cmp_pm.sort_values(by=['TRIP_ID_EXTERNAL', 'Date', 'VEHICLE_ID', 'Open_Time']).reset_index()

In [31]:
# # Transit Speeds on CMP Segment 
def match_intermediate_apc_stops(apc_pairs, apc_cmp, overlap_pairs, cmp_segs_prj, timep):
    # ### Auto Matched Pairs 
    apc_pairs['cmp_segid'] = apc_pairs['cmp_segid'].astype(int)
    apc_pairs['cur_stop_id'] = apc_pairs['cur_stop_id'].astype(int)
    apc_pairs['next_stop_id'] = apc_pairs['next_stop_id'].astype(int)
    
    # ### Manually Matched Pairs 
    pair_cnt = len(apc_pairs)
    for cur_stop_idx in range(len(overlap_pairs)):
        cur_stopid = overlap_pairs.loc[cur_stop_idx, 'pre_stopid']
        next_stopid = overlap_pairs.loc[cur_stop_idx, 'next_stopid']
        cur_stop_trips = apc_cmp.index[apc_cmp['BS_ID']==cur_stopid].tolist()
        for cur_stop_trip_idx in cur_stop_trips:
            if apc_cmp.loc[cur_stop_trip_idx + 1, 'BS_ID'] == next_stopid:
    
                cur_stop_trip_id = apc_cmp.loc[cur_stop_trip_idx, 'TRIP_ID_EXTERNAL']
                cur_stop_date = apc_cmp.loc[cur_stop_trip_idx, 'Date']
                cur_stop_veh_id = apc_cmp.loc[cur_stop_trip_idx, 'VEHICLE_ID']
                cur_stop_route_alpha = apc_cmp.loc[cur_stop_trip_idx, 'ROUTE_ALPHA']
                cur_stop_route_dir = apc_cmp.loc[cur_stop_trip_idx, 'DIRECTION']
                cur_stop_open_time = apc_cmp.loc[cur_stop_trip_idx, 'Open_Time']
                cur_stop_close_time = apc_cmp.loc[cur_stop_trip_idx, 'Close_Time']
                cur_stop_dwell_time = apc_cmp.loc[cur_stop_trip_idx, 'DWELL_TIME']
    
                next_stop_trip_id = apc_cmp.loc[cur_stop_trip_idx + 1, 'TRIP_ID_EXTERNAL']
                next_stop_date = apc_cmp.loc[cur_stop_trip_idx + 1, 'Date']
                next_stop_veh_id = apc_cmp.loc[cur_stop_trip_idx + 1, 'VEHICLE_ID']
                next_stop_open_time = apc_cmp.loc[cur_stop_trip_idx + 1, 'Open_Time']
                next_stop_close_time = apc_cmp.loc[cur_stop_trip_idx + 1, 'Close_Time']
                next_stop_dwell_time = apc_cmp.loc[cur_stop_trip_idx + 1, 'DWELL_TIME']
    
                # Check if two stops share the same trip id, date, and vehicle id
                if (cur_stop_trip_id == next_stop_trip_id) & (cur_stop_date == next_stop_date) & (cur_stop_veh_id == next_stop_veh_id):
                    cur_next_traveltime = (next_stop_open_time-cur_stop_open_time).total_seconds()
                    cur_next_loc_dis = overlap_pairs.loc[cur_stop_idx, 'cmp_overlap_len']/5280/overlap_pairs.loc[cur_stop_idx, 'cmp_overlap_ratio']
                    
                    # Remove records deemed erroneous according to memo from last cycle
                    if (cur_next_loc_dis>0) & (cur_next_traveltime>0):
                        if (3600 * cur_next_loc_dis/cur_next_traveltime<=55) | (overlap_pairs.loc[cur_stop_idx, 'cmp_segid']==237):
                            # Add matched stop pairs to the dataframe for subsequent speed calculation
                            apc_pairs.loc[pair_cnt, 'cmp_segid'] = overlap_pairs.loc[cur_stop_idx, 'cmp_segid']
                            apc_pairs.loc[pair_cnt, 'trip_id'] = cur_stop_trip_id
                            apc_pairs.loc[pair_cnt, 'trip_date'] = cur_stop_date
                            apc_pairs.loc[pair_cnt, 'vehicle_id'] = cur_stop_veh_id
                            apc_pairs.loc[pair_cnt, 'route_alpha'] = cur_stop_route_alpha
                            apc_pairs.loc[pair_cnt, 'direction'] = cur_stop_route_dir
    
                            apc_pairs.loc[pair_cnt, 'cur_stop_id'] = cur_stopid
                            apc_pairs.loc[pair_cnt, 'cur_stop_open_time'] = cur_stop_open_time
                            apc_pairs.loc[pair_cnt, 'cur_stop_close_time'] = cur_stop_close_time
                            apc_pairs.loc[pair_cnt, 'cur_stop_dwell_time'] = cur_stop_dwell_time
    
                            apc_pairs.loc[pair_cnt, 'next_stop_id'] = next_stopid
                            apc_pairs.loc[pair_cnt, 'next_stop_open_time'] = next_stop_open_time
                            apc_pairs.loc[pair_cnt, 'next_stop_close_time'] = next_stop_close_time
                            apc_pairs.loc[pair_cnt, 'next_stop_dwell_time'] = next_stop_dwell_time
                            
                            apc_pairs.loc[pair_cnt, 'cur_next_time'] = cur_next_traveltime
                            apc_pairs.loc[pair_cnt, 'cur_next_loc_dis'] = cur_next_loc_dis
                            apc_pairs.loc[pair_cnt, 'cur_next_rev_dis'] = apc_cmp.loc[cur_stop_trip_idx, 'REV_DISTANCE']
    
                        pair_cnt = pair_cnt + 1
                  
    apc_pairs['cur_next_loc_dis'] = np.where(apc_pairs['cur_next_loc_dis'] >= apc_pairs['cur_next_rev_dis'], 
                                            apc_pairs['cur_next_loc_dis'],
                                            apc_pairs['cur_next_rev_dis'])
    
    apc_pairs_clean = apc_pairs[apc_pairs['cur_stop_dwell_time']<180]
    apc_pairs_clean['route_alpha'] = apc_pairs_clean['route_alpha'].apply(lambda x: str(x).lstrip("0"))  #remove leading zeros
    apc_pairs_clean['route_dir'] = apc_pairs_clean['route_alpha'].astype(str)  + '_' + apc_pairs_clean['direction']
    
    grpby_cols = ['cmp_segid', 'trip_id', 'trip_date', 'route_dir']
    apc_trip_speeds = apc_pairs_clean.groupby(grpby_cols).agg({'cur_next_loc_dis': 'sum',
                                                                'cur_next_time': 'sum'}).reset_index()
    
    apc_trip_speeds.columns = grpby_cols + ['trip_stop_distance', 'trip_traveltime']
    apc_trip_speeds['trip_loc_speed'] = 3600* apc_trip_speeds['trip_stop_distance']/apc_trip_speeds['trip_traveltime']
    
    apc_trip_speeds = pd.merge(apc_trip_speeds, cmp_segs_prj[['cmp_segid', 'length']], on='cmp_segid', how='left')
    apc_trip_speeds['len_ratio'] = 100*apc_trip_speeds['trip_stop_distance']/apc_trip_speeds['length']
    
    #apc_trip_speeds.to_csv(os.path.join(OUT_DIR, 'CMP2021_APC_Matched_Trips_%s.csv' %timep), index=False)
    
    # Only include trips covering at least 50% of CMP length
    apc_trip_speeds_over50 = apc_trip_speeds[apc_trip_speeds['len_ratio']>=50]
    apc_cmp_speeds = apc_trip_speeds_over50.groupby(['cmp_segid']).agg({'trip_loc_speed': ['mean', 'std'],
                                                                        'trip_id': 'count'}).reset_index()
    
    apc_cmp_speeds.columns = ['cmp_segid', 'avg_speed', 'std_dev', 'sample_size']
    apc_cmp_speeds['cov'] = 100* apc_cmp_speeds['std_dev']/apc_cmp_speeds['avg_speed']
    
    apc_cmp_speeds['year'] = 2021
    apc_cmp_speeds['source'] = 'APC'
    apc_cmp_speeds['period'] = timep
    
    apc_cmp_speeds_routes = apc_trip_speeds_over50.groupby(['cmp_segid'])['route_dir'].agg(['unique']).reset_index()
    apc_cmp_speeds_routes.columns = ['cmp_segid', 'comment']
    apc_cmp_speeds = apc_cmp_speeds.merge(apc_cmp_speeds_routes, on='cmp_segid', how='left')
    
    return apc_cmp_speeds
    
# ## AM 
apc_cmp_speeds_am = match_intermediate_apc_stops(apc_pairs_am, apc_cmp_am, overlap_pairs, cmp_segs_prj, 'AM')

# ## PM 
apc_cmp_speeds_pm = match_intermediate_apc_stops(apc_pairs_pm, apc_cmp_pm, overlap_pairs, cmp_segs_prj, 'PM')

In [68]:
# # Transit Speeds on CMP Segment 
def match_intermediate_apc_stops2(apc_pairs, apc_cmp, overlap_pairs, cmp_segs_prj, timep):
    # ### Auto Matched Pairs 
    apc_pairs['cmp_segid'] = apc_pairs['cmp_segid'].astype(int)
    apc_pairs['cur_stop_id'] = apc_pairs['cur_stop_id'].astype(int)
    apc_pairs['next_stop_id'] = apc_pairs['next_stop_id'].astype(int)
    
    # ### Manually Matched Pairs 
    pair_cnt = len(apc_pairs)
    for cur_stop_idx in range(len(overlap_pairs)):
        cur_stopid = overlap_pairs.loc[cur_stop_idx, 'pre_stopid']
        next_stopid = overlap_pairs.loc[cur_stop_idx, 'next_stopid']
        cur_stop_trips = apc_cmp.index[apc_cmp['BS_ID']==cur_stopid].tolist()
        for cur_stop_trip_idx in cur_stop_trips:
            if apc_cmp.loc[cur_stop_trip_idx + 1, 'BS_ID'] == next_stopid:
    
                cur_stop_trip_id = apc_cmp.loc[cur_stop_trip_idx, 'TRIP_ID_EXTERNAL']
                cur_stop_date = apc_cmp.loc[cur_stop_trip_idx, 'Date']
                cur_stop_veh_id = apc_cmp.loc[cur_stop_trip_idx, 'VEHICLE_ID']
                cur_stop_route_alpha = apc_cmp.loc[cur_stop_trip_idx, 'ROUTE_ALPHA']
                cur_stop_route_dir = apc_cmp.loc[cur_stop_trip_idx, 'DIRECTION']
                cur_stop_open_time = apc_cmp.loc[cur_stop_trip_idx, 'Open_Time']
                cur_stop_close_time = apc_cmp.loc[cur_stop_trip_idx, 'Close_Time']
                cur_stop_dwell_time = apc_cmp.loc[cur_stop_trip_idx, 'DWELL_TIME']
    
                next_stop_trip_id = apc_cmp.loc[cur_stop_trip_idx + 1, 'TRIP_ID_EXTERNAL']
                next_stop_date = apc_cmp.loc[cur_stop_trip_idx + 1, 'Date']
                next_stop_veh_id = apc_cmp.loc[cur_stop_trip_idx + 1, 'VEHICLE_ID']
                next_stop_open_time = apc_cmp.loc[cur_stop_trip_idx + 1, 'Open_Time']
                next_stop_close_time = apc_cmp.loc[cur_stop_trip_idx + 1, 'Close_Time']
                next_stop_dwell_time = apc_cmp.loc[cur_stop_trip_idx + 1, 'DWELL_TIME']
    
                # Check if two stops share the same trip id, date, and vehicle id
                if (cur_stop_trip_id == next_stop_trip_id) & (cur_stop_date == next_stop_date) & (cur_stop_veh_id == next_stop_veh_id):
                    cur_next_traveltime = (next_stop_open_time-cur_stop_open_time).total_seconds()
                    cur_next_loc_dis = overlap_pairs.loc[cur_stop_idx, 'cmp_overlap_len']/5280/overlap_pairs.loc[cur_stop_idx, 'cmp_overlap_ratio']
                    
                    # Remove records deemed erroneous according to memo from last cycle
                    if (cur_next_loc_dis>0) & (cur_next_traveltime>0):
                        if 3600 * cur_next_loc_dis/cur_next_traveltime<=55:
                            # Add matched stop pairs to the dataframe for subsequent speed calculation
                            apc_pairs.loc[pair_cnt, 'cmp_segid'] = overlap_pairs.loc[cur_stop_idx, 'cmp_segid']
                            apc_pairs.loc[pair_cnt, 'trip_id'] = cur_stop_trip_id
                            apc_pairs.loc[pair_cnt, 'trip_date'] = cur_stop_date
                            apc_pairs.loc[pair_cnt, 'vehicle_id'] = cur_stop_veh_id
                            apc_pairs.loc[pair_cnt, 'route_alpha'] = cur_stop_route_alpha
                            apc_pairs.loc[pair_cnt, 'direction'] = cur_stop_route_dir
    
                            apc_pairs.loc[pair_cnt, 'cur_stop_id'] = cur_stopid
                            apc_pairs.loc[pair_cnt, 'cur_stop_open_time'] = cur_stop_open_time
                            apc_pairs.loc[pair_cnt, 'cur_stop_close_time'] = cur_stop_close_time
                            apc_pairs.loc[pair_cnt, 'cur_stop_dwell_time'] = cur_stop_dwell_time
    
                            apc_pairs.loc[pair_cnt, 'next_stop_id'] = next_stopid
                            apc_pairs.loc[pair_cnt, 'next_stop_open_time'] = next_stop_open_time
                            apc_pairs.loc[pair_cnt, 'next_stop_close_time'] = next_stop_close_time
                            apc_pairs.loc[pair_cnt, 'next_stop_dwell_time'] = next_stop_dwell_time
                            
                            apc_pairs.loc[pair_cnt, 'cur_next_time'] = cur_next_traveltime
                            apc_pairs.loc[pair_cnt, 'cur_next_loc_dis'] = cur_next_loc_dis
                            apc_pairs.loc[pair_cnt, 'cur_next_rev_dis'] = apc_cmp.loc[cur_stop_trip_idx, 'REV_DISTANCE']
    
                        pair_cnt = pair_cnt + 1  
    apc_pairs['cur_next_loc_dis'] = np.where(apc_pairs['cur_next_loc_dis'] >= apc_pairs['cur_next_rev_dis'], 
                                            apc_pairs['cur_next_loc_dis'],
                                            apc_pairs['cur_next_rev_dis'])
    return apc_pairs

In [9]:
cur_stop_trips = apc_cmp_am.index[apc_cmp_am['BS_ID']==7948].tolist()

NameError: name 'apc_cmp_am' is not defined

In [274]:
len(cur_stop_trips)

191

In [ ]:
y = apc_cmp_am.index[(apc_cmp_am['BS_ID']==7948)].tolist()

In [271]:
cur_stop_trips = [x+1 for x in cur_stop_trips]

In [272]:
apc_cmp_am.loc[cur_stop_trips, 'BS_ID'].unique()

array([7948, 8017, 8019, 3229, 8021, 8020, 4936, 7558, 4934, 4935, 8018,
       5166, 3670], dtype=int64)

In [232]:
apc_cmp_am.loc[(apc_cmp_am.index.isin(cur_stop_trips)) & (apc_cmp_am['BS_ID']==8017),'Open_Time']

Series([], Name: Open_Time, dtype: datetime64[ns])

In [276]:
k = cur_stop_trips[10]
print(apc_cmp_am.loc[k+1, 'BS_ID'])
print(apc_cmp_am.loc[k-1, 'BS_ID'])

7948
7948


In [14]:
# apc_cmp_am[apc_cmp_am['ROUTE_ALPHA'].isin([25,'025'])].to_csv('Muni25_PreCOVID_2022.csv', index=False)
apc_cmp_am[apc_cmp_am['ROUTE_ALPHA'].isin([25,'025'])].to_csv('Muni25_05_2022.csv', index=False)

In [235]:
cur_stop_idx = 21
cur_stopid = overlap_pairs.loc[cur_stop_idx, 'pre_stopid']
next_stopid = overlap_pairs.loc[cur_stop_idx, 'next_stopid']

In [236]:
print(cur_stopid)
print(next_stopid)

8018
7948


In [191]:
cur_stop_trips = apc_cmp_am.index[apc_cmp_am['BS_ID']==cur_stopid].tolist()
for cur_stop_trip_idx in cur_stop_trips:
    if apc_cmp_am.loc[cur_stop_trip_idx + 1, 'BS_ID'] == next_stopid:
        print(apc_cmp_am.loc[cur_stop_trip_idx + 1,['BS_ID', 'Open_Time']])

BS_ID                       7948
Open_Time    2021-04-27 08:33:14
Name: 135474, dtype: object
BS_ID                       7948
Open_Time    2021-04-22 07:58:34
Name: 135498, dtype: object
BS_ID                       7948
Open_Time    2021-05-05 07:15:26
Name: 135545, dtype: object
BS_ID                       7948
Open_Time    2021-04-08 06:56:09
Name: 135552, dtype: object
BS_ID                       7948
Open_Time    2021-04-27 06:57:08
Name: 135561, dtype: object
BS_ID                       7948
Open_Time    2021-05-19 08:28:36
Name: 278214, dtype: object
BS_ID                       7948
Open_Time    2021-05-19 08:04:28
Name: 278218, dtype: object
BS_ID                       7948
Open_Time    2021-05-19 06:59:03
Name: 278250, dtype: object


In [193]:
apc_cmp_am.columns

Index(['index', 'ACTUALDATE', 'SERVICE_DAY_OF_WEEK', 'PRIMARY_KEY',
       'EFFECTIVE_DATE_KEY', 'TRAP_SIGN_ID', 'BOOKING_ID', 'TRAP_PATTERN_NAME',
       'BLOCK_NUM_EXTERNAL', 'TRAP_BUS_TYPE_ID', 'ROUTE_ALPHA', 'DIRECTION',
       'TRIP_ID_EXTERNAL', 'START_TIME', 'END_TIME', 'TRIP_START_TIME',
       'TRIP_START_HOUR', 'SCHED_RUNNING_TIME_SECONDS', 'TOTAL_TRIP_DISTANCE',
       'STOP_LINE_SEQUENCE', 'BS_ID', 'IS_TERMINAL_INITIAL',
       'ROUTE_SEGMENT_ID', 'ACCUM_DISTANCE', 'SCHEDULE_LOAD_DATE',
       'APC_SOURCE', 'VEHICLE_ID', 'ACTUAL_SEQUENCE', 'RUN_ID', 'OPERATOR_ID',
       'STOP_ARRIVAL_HOUR', 'OPEN_DATE_TIME', 'CLOSE_DATE_TIME', 'DWELL_TIME',
       'ONS', 'OFFS', 'MAX_LOAD', 'REV_DISTANCE', 'NON_REV_DISTANCE',
       'REV_SECONDS', 'NON_REV_SECONDS', 'PASSENGER_MILES', 'PASSENGER_HOURS',
       'VEH_LAT', 'VEH_LONG', 'IMPORT_ERROR', 'IMPORT_TRIP_ERROR',
       'CALC_SPEED', 'IS_TRIP_SAMPLED', 'STOPS_SAMPLED', 'ACTIVITY_LOAD_DATE',
       'Date', 'DOW', 'DayType', 'Month', '

In [201]:
apc_cmp_am['IS_TERMINAL_INITIAL'].unique()

array(['INITIAL', 'INTRANSIT', 'TERMINAL'], dtype=object)

In [202]:
apc_cmp_am.loc[(apc_cmp_am['BS_ID']==7948) & (apc_cmp_am['IS_TERMINAL_INITIAL']=='INITIAL'),'TRIP_ID_EXTERNAL']

135457    9765352
135458    9765352
135459    9765353
135460    9765353
135461    9765353
135462    9765353
135463    9765354
135467    9765354
135471    9765354
135474    9765354
135478    9765354
135482    9765354
135486    9765355
135491    9765355
135496    9765356
135498    9765356
135501    9765356
135503    9765356
135506    9765357
135509    9765357
135513    9765357
135518    9765357
135524    9765357
135527    9765357
135531    9765358
135534    9765358
135538    9765359
135543    9765359
135545    9765359
135549    9765359
135552    9765385
135556    9765385
135559    9765385
135561    9765385
135566    9765385
135573    9765407
278209    9883398
278210    9883398
278211    9883399
278214    9883399
278216    9883399
278218    9883401
278222    9883401
278227    9883402
278231    9883402
278234    9883402
278238    9883404
278243    9883404
278248    9883430
278250    9883430
278253    9883430
Name: TRIP_ID_EXTERNAL, dtype: object

In [205]:
apc_cmp_am.loc[apc_cmp_am['TRIP_ID_EXTERNAL']==9883430, ['BS_ID','ROUTE_ALPHA', 'DIRECTION','ACTUAL_SEQUENCE']]

,BS_ID,ROUTE_ALPHA,DIRECTION,ACTUAL_SEQUENCE
278248,7948,25,OUTBOUND,2.0
278249,8019,25,OUTBOUND,3.0
278250,7948,25,OUTBOUND,2.0
278251,8017,25,OUTBOUND,3.0
278252,8021,25,OUTBOUND,4.0
278253,7948,25,OUTBOUND,2.0
278254,8019,25,OUTBOUND,3.0
278255,3229,25,OUTBOUND,4.0
278256,7558,25,OUTBOUND,5.0


In [192]:
cur_stop_trips = apc_cmp_am.index[apc_cmp_am['BS_ID']==cur_stopid].tolist()
for cur_stop_trip_idx in cur_stop_trips:
    if apc_cmp_am.loc[cur_stop_trip_idx + 1, 'BS_ID'] == next_stopid:
        print(apc_cmp_am.loc[cur_stop_trip_idx,['BS_ID', 'Close_Time']])

BS_ID                        8019
Close_Time    2021-04-20 08:54:41
Name: 135473, dtype: object
BS_ID                        8019
Close_Time    2021-04-14 08:23:25
Name: 135497, dtype: object
BS_ID                        8019
Close_Time    2021-04-22 07:42:16
Name: 135544, dtype: object
BS_ID                        8019
Close_Time    2021-05-06 07:37:57
Name: 135551, dtype: object
BS_ID                        8019
Close_Time    2021-04-20 07:25:18
Name: 135560, dtype: object
BS_ID                        8019
Close_Time    2021-05-18 08:56:42
Name: 278213, dtype: object
BS_ID                        8019
Close_Time    2021-05-20 08:55:31
Name: 278217, dtype: object
BS_ID                        8019
Close_Time    2021-05-18 07:25:43
Name: 278249, dtype: object


In [ ]:
cur_stop_trips = apc_cmp_am.index[apc_cmp_am['BS_ID']==cur_stopid].tolist()
for cur_stop_trip_idx in cur_stop_trips:
    if apc_cmp_am.loc[cur_stop_trip_idx + 1, 'BS_ID'] == next_stopid:
        print(apc_cmp_am.loc[cur_stop_trip_idx + 1,'Open_Time'])

In [179]:
test_am = match_intermediate_apc_stops2(apc_pairs_am, apc_cmp_am, overlap_pairs, cmp_segs_prj, 'AM')

In [183]:
overlap_pairs

,cmp_segid,pre_stopid,next_stopid,cmp_overlap_len,cmp_overlap_ratio,Note,Unnamed: 6
0,194,7301,5817,670.00000,0.595027,NaN,Y
1,195,7301,5817,456.00000,0.404973,NaN,Y
2,204,5129,6431,2964.00000,0.739336,NaN,AM
3,205,6434,7624,1001.00000,0.481713,NaN,N
4,205,7624,5130,3661.00000,0.663585,NaN,N
5,25,7758,5950,4895.00000,1.000000,NaN,Y
6,181,6045,4823,1417.00000,0.762238,NaN,Y
7,41,4017,4346,621.00000,0.646202,NaN,Y
8,11,3166,4969,1273.00000,0.930556,NaN,Y
9,123,4242,5645,2374.00000,0.733169,NaN,AM


In [180]:
apc_pairs = test_am

In [181]:
apc_pairs['cur_next_loc_dis'] = np.where(apc_pairs['cur_next_loc_dis'] >= apc_pairs['cur_next_rev_dis'], 
                                            apc_pairs['cur_next_loc_dis'],
                                            apc_pairs['cur_next_rev_dis'])
    
apc_pairs_clean = apc_pairs[(apc_pairs['cur_stop_dwell_time']<180) | (apc_pairs['cmp_segid'].isin([237,245]))]
apc_pairs_clean['route_alpha'] = apc_pairs_clean['route_alpha'].apply(lambda x: str(x).lstrip("0"))  #remove leading zeros
apc_pairs_clean['route_dir'] = apc_pairs_clean['route_alpha'].astype(str)  + '_' + apc_pairs_clean['direction']

In [182]:
apc_pairs_clean[apc_pairs_clean['cmp_segid'].isin([237])]

,cmp_segid,trip_id,trip_date,vehicle_id,route_alpha,direction,cur_stop_id,cur_stop_open_time,cur_stop_close_time,cur_stop_dwell_time,cur_stop_loc,next_stop_id,next_stop_open_time,next_stop_close_time,next_stop_dwell_time,next_stop_loc,cur_next_time,cur_next_loc_dis,cur_next_rev_dis,route_dir


In [88]:
grpby_cols = ['cmp_segid', 'trip_id', 'trip_date', 'route_dir']
apc_trip_speeds = apc_pairs_clean.groupby(grpby_cols).agg({'cur_next_loc_dis': 'sum',
                                                            'cur_next_time': 'sum'}).reset_index()

apc_trip_speeds.columns = grpby_cols + ['trip_stop_distance', 'trip_traveltime']
apc_trip_speeds['trip_loc_speed'] = 3600* apc_trip_speeds['trip_stop_distance']/apc_trip_speeds['trip_traveltime']

apc_trip_speeds = pd.merge(apc_trip_speeds, cmp_segs_prj[['cmp_segid', 'length']], on='cmp_segid', how='left')
apc_trip_speeds['len_ratio'] = 100*apc_trip_speeds['trip_stop_distance']/apc_trip_speeds['length']

In [90]:
apc_pairs_clean[apc_pairs_clean['cmp_segid'].isin([237,245])]

,cmp_segid,trip_id,trip_date,vehicle_id,route_alpha,direction,cur_stop_id,cur_stop_open_time,cur_stop_close_time,cur_stop_dwell_time,cur_stop_loc,next_stop_id,next_stop_open_time,next_stop_close_time,next_stop_dwell_time,next_stop_loc,cur_next_time,cur_next_loc_dis,cur_next_rev_dis,route_dir
110274,245.0,9765354,2021-04-20,8837.0,25,OUTBOUND,7948.0,2021-04-20 08:27:34,2021-04-20 08:44:26,1012.0,NaN,8017.0,2021-04-20 08:52:56,2021-04-20 08:53:06,10.0,NaN,1522.0,3.548174,3.548174,25_OUTBOUND
110275,245.0,9765355,2021-04-20,8824.0,25,OUTBOUND,7948.0,2021-04-20 08:13:43,2021-04-20 08:29:18,935.0,NaN,8017.0,2021-04-20 08:39:53,2021-04-20 08:40:21,28.0,NaN,1570.0,3.548174,3.548174,25_OUTBOUND
110276,245.0,9765355,2021-05-11,8826.0,25,OUTBOUND,7948.0,2021-05-11 08:13:52,2021-05-11 08:28:59,907.0,NaN,8017.0,2021-05-11 08:37:49,2021-05-11 08:38:11,22.0,NaN,1437.0,3.548174,3.548174,25_OUTBOUND
110277,245.0,9765357,2021-04-13,8844.0,25,OUTBOUND,7948.0,2021-04-13 07:42:45,2021-04-13 07:58:58,973.0,NaN,8017.0,2021-04-13 08:07:33,2021-04-13 08:07:42,9.0,NaN,1488.0,3.548174,3.548174,25_OUTBOUND
110278,245.0,9765357,2021-04-20,8837.0,25,OUTBOUND,7948.0,2021-04-20 07:40:47,2021-04-20 07:58:56,1089.0,NaN,8017.0,2021-04-20 08:08:02,2021-04-20 08:08:13,11.0,NaN,1635.0,3.548174,3.548174,25_OUTBOUND
110279,245.0,9765357,2021-04-27,8876.0,25,OUTBOUND,7948.0,2021-04-27 07:43:59,2021-04-27 08:02:04,1085.0,NaN,8017.0,2021-04-27 08:11:01,2021-04-27 08:11:10,9.0,NaN,1622.0,3.548174,3.548174,25_OUTBOUND
110280,245.0,9765357,2021-04-28,8887.0,25,OUTBOUND,7948.0,2021-04-28 07:42:25,2021-04-28 07:59:37,1032.0,NaN,8017.0,2021-04-28 08:08:16,2021-04-28 08:08:28,12.0,NaN,1551.0,3.548174,3.548174,25_OUTBOUND
110281,245.0,9765359,2021-05-05,8879.0,25,OUTBOUND,7948.0,2021-05-05 07:15:26,2021-05-05 07:29:53,867.0,NaN,8017.0,2021-05-05 07:39:02,2021-05-05 07:39:13,11.0,NaN,1416.0,3.548174,3.548174,25_OUTBOUND
110282,245.0,9765359,2021-05-06,8868.0,25,OUTBOUND,7948.0,2021-05-06 07:08:03,2021-05-06 07:28:49,1246.0,NaN,8017.0,2021-05-06 07:36:14,2021-05-06 07:36:37,23.0,NaN,1691.0,3.548174,3.548174,25_OUTBOUND
110283,245.0,9765385,2021-04-27,8876.0,25,OUTBOUND,7948.0,2021-04-27 06:57:08,2021-04-27 07:14:39,1051.0,NaN,8017.0,2021-04-27 07:23:43,2021-04-27 07:23:52,9.0,NaN,1595.0,3.548174,3.548174,25_OUTBOUND


In [35]:
cur_stop_idx=22
overlap_pairs.loc[cur_stop_idx, 'pre_stopid']

7948

In [49]:
apc_cmp.loc[1694265, ]

ACTUALDATE              06-APR-21 00:00:00
SERVICE_DAY_OF_WEEK                WEEKDAY
PRIMARY_KEY                      215883413
EFFECTIVE_DATE_KEY                    3748
TRAP_SIGN_ID                           125
                              ...         
Open_Period                             PM
Open_Time              2021-04-06 21:56:23
Day                                      6
Open_Time_float                   9.939722
Close_Time_float                 10.000833
Name: 1694265, Length: 68, dtype: object

In [32]:
apc_cmp_speeds_am[apc_cmp_speeds_am['cmp_segid'].isin([237,245])]

,cmp_segid,avg_speed,std_dev,sample_size,cov,year,source,period,comment


In [64]:
apc_cmp_speeds_pm['cmp_segid'].unique()

array([  4.,   5.,   6.,   7.,   8.,   9.,  15.,  16.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  28.,  29.,  30.,  38.,  39.,  48.,  60.,
        61.,  62.,  63.,  64.,  65.,  66.,  72.,  74.,  75.,  91.,  92.,
        96., 103., 104., 105., 106., 107., 108., 109., 110., 111., 112.,
       113., 114., 115., 116., 117., 118., 119., 120., 121., 127., 130.,
       133., 134., 138., 141., 143., 144., 145., 146., 149., 152., 153.,
       154., 155., 156., 157., 158., 159., 160., 161., 163., 166., 167.,
       168., 169., 170., 171., 172., 173., 174., 175., 176., 177., 178.,
       180., 181., 183., 188., 189., 190., 191., 194., 195., 200., 201.,
       202., 203., 222., 223., 224., 225., 226., 227., 230., 231., 246.,
       247., 248., 249., 254., 255., 256., 257., 258., 259., 260., 261.,
       262., 263.])